# WIP MLflow Client API

| 🚨 Consider a WIP until this message goes away... 🚨 |
|-|

## MlflowClient

`MlflowClient` is a MLflow client that uses two other MLflow clients:

| MLflow Client | Description |
|-|-|
| `TrackingServiceClient` | Client of a [MLflow Tracking Server](https://mlflow.org/docs/latest/self-hosting/architecture/tracking-server/) that creates and manages experiments and runs. |
| `TracingClient` | Client of a [MLflow Registry Server](https://mlflow.org/docs/latest/ml/model-registry/) that creates and manages registered models and model versions. |

### TrackingServiceClient

`TrackingServiceClient` is a MLflow client of a MLflow Tracking Server to create and manage experiments and runs.

In [0]:
%skip

from mlflow.tracking.client import TrackingServiceClient

tracking_service_client = TrackingServiceClient(tracking_uri="http://localhost:5000")

## Databricks

Databricks users are supposed to use `mlflow` methods (e.g., `mlflow.search_experiments`).

See [mlflow/\_\_init__.py](https://github.com/mlflow/mlflow/blob/v3.8.1/mlflow/__init__.py#L234)

## Install MLflow Client

In [0]:
%pip install -qU mlflow>=3.8.1
%restart_python

In [0]:
import mlflow
print(f"MLflow version: {mlflow.__version__}")

# to be notified about a possible upgrade in the meantime
assert mlflow.__version__ == "3.8.1"

In [0]:
assert mlflow.tracking.get_tracking_uri() == "databricks"

In [0]:
assert mlflow.tracking.get_registry_uri() == "databricks-uc"

In [0]:
from mlflow import MlflowClient

client = MlflowClient()

## List Training Runs

Experiment type: **GenAI apps & agents**

![Training runs in Databricks UI](./databricks-experiments-training-runs.png)

In [0]:
%skip

# Does not work and gives None :(
experiment = client.get_experiment_by_name(name="Demo Everyday English Editor")
print(experiment)

In [0]:
%skip

%run "../DSPy/Demo Everyday English Editor"

In [0]:
import rich

experiment = client.search_experiments(filter_string="name ILIKE '%Demo Everyday English Editor%'")[0]
rich.print(experiment)

In [0]:
for tag_key, tag_value in experiment.tags.items():
    print(tag_key, tag_value)

In [0]:
%skip It appears that the mlflow system tags cannot be used for searching.
import rich

for experiment in client.search_experiments(filter_string='tags.mlflow.ownerEmail = "jacek@japila.pl"'):
    rich.print(experiment)

In [0]:
import rich

for exp_run in client.search_runs(experiment_ids=experiment.experiment_id):
    rich.print(exp_run.info.run_name, "\t", exp_run.data.tags.get('mlflow.parentRunId', 'N/A'))

In [0]:
import mlflow

display(mlflow.search_runs(experiment_ids=experiment.experiment_id))

In [0]:
client.search_logged_models(experiment_ids=[experiment.experiment_id])

## MLflow Tracing SDK

MLflow Tracing SDK ([mlflow-tracing](https://pypi.org/project/mlflow-tracing/)) is an open-source, lightweight Python package that only includes the minimum set of dependencies and functionality to instrument your code/models/agents with MLflow Tracing.

See [mlflow/\_\_init__.py](https://github.com/mlflow/mlflow/blob/v3.8.1/mlflow/__init__.py#L197-L232)

In [0]:
import mlflow

import dbruntime.databricks_repl_context as repl_ctx
current_user = repl_ctx.get_context().user

experiment = mlflow.set_experiment(f"/Users/{current_user}/Hello MLflow Tracing")
experiment_id = experiment.experiment_id

import rich
rich.print(experiment)

### Tracing API

* `add_trace`
* `delete_trace_tag`
* `flush_trace_async_logging`
* `get_active_trace_id`
* `get_current_active_span`
* `get_last_active_trace_id`
* `get_trace`
* `log_trace`
* `search_traces`
* `set_trace_tag`
* `start_span`
* `start_span_no_context`
* `trace`
* `update_current_trace`

### Manual Tracing

[Manual Tracing](https://mlflow.org/docs/latest/genai/tracing/app-instrumentation/manual-tracing/)

In [0]:
import mlflow


@mlflow.trace(span_type="func", attributes={"key": "value"})
def add_1(x):
    return x + 1


@mlflow.trace(span_type="func", attributes={"key1": "value1"})
def minus_1(x):
    return x - 1


@mlflow.trace(name="Trace Test")
def trace_test(x):
    step1 = add_1(x)
    return minus_1(step1)


trace_test(4)

### Assessment API

* `get_assessment`
* `delete_assessment`
* `log_assessment`
* `update_assessment`
* `log_expectation`
* `log_feedback`
* `override_feedback`